In [3]:
import sys
import os
import json
import xml.etree.ElementTree as ET

In [4]:



START_BOUNDING_BOX_ID = 1
PRE_DEFINE_CATEGORIES = {}
# If necessary, pre-define category and its id
#  PRE_DEFINE_CATEGORIES = {"aeroplane": 1, "bicycle": 2, "bird": 3, "boat": 4,
                         #  "bottle":5, "bus": 6, "car": 7, "cat": 8, "chair": 9,
                         #  "cow": 10, "diningtable": 11, "dog": 12, "horse": 13,
                         #  "motorbike": 14, "person": 15, "pottedplant": 16,
                         #  "sheep": 17, "sofa": 18, "train": 19, "tvmonitor": 20}


def get(root, name):
    vars = root.findall(name)
    return vars


def get_and_check(root, name, length):
    vars = root.findall(name)
    if len(vars) == 0:
        raise NotImplementedError('Can not find %s in %s.'%(name, root.tag))
    if length > 0 and len(vars) != length:
        raise NotImplementedError('The size of %s is supposed to be %d, but is %d.'%(name, length, len(vars)))
    if length == 1:
        vars = vars[0]
    return vars


def get_filename_as_int(filename):
    try:
        filename = os.path.splitext(filename)[0]
        return int(filename)
    except:
        raise NotImplementedError('Filename %s is supposed to be an integer.'%(filename))


def convert(xml_list, xml_dir, json_file):
    list_fp = open(xml_list, 'r')
    json_dict = {"images":[], "type": "instances", "annotations": [],
                 "categories": []}
    categories = PRE_DEFINE_CATEGORIES
    bnd_id = START_BOUNDING_BOX_ID
    for line in list_fp:
        line = line.strip()
        print("Processing %s"%(line))
        xml_f = os.path.join(xml_dir, line)
        tree = ET.parse(xml_f)
        root = tree.getroot()
        path = get(root, 'path')
        if len(path) == 1:
            filename = os.path.basename(path[0].text)
        elif len(path) == 0:
            filename = get_and_check(root, 'filename', 1).text
        else:
            raise NotImplementedError('%d paths found in %s'%(len(path), line))
        ## The filename must be a number
        image_id = get_filename_as_int(filename)
        size = get_and_check(root, 'size', 1)
        width = int(get_and_check(size, 'width', 1).text)
        height = int(get_and_check(size, 'height', 1).text)
        image = {'file_name': filename, 'height': height, 'width': width,
                 'id':image_id}
        json_dict['images'].append(image)
        ## Cruuently we do not support segmentation
        #  segmented = get_and_check(root, 'segmented', 1).text
        #  assert segmented == '0'
        for obj in get(root, 'object'):
            category = get_and_check(obj, 'name', 1).text
            if category not in categories:
                new_id = len(categories)
                categories[category] = new_id
            category_id = categories[category]
            bndbox = get_and_check(obj, 'bndbox', 1)
            xmin = int(get_and_check(bndbox, 'xmin', 1).text) - 1
            ymin = int(get_and_check(bndbox, 'ymin', 1).text) - 1
            xmax = int(get_and_check(bndbox, 'xmax', 1).text)
            ymax = int(get_and_check(bndbox, 'ymax', 1).text)
            assert(xmax > xmin)
            assert(ymax > ymin)
            o_width = abs(xmax - xmin)
            o_height = abs(ymax - ymin)
            ann = {'area': o_width*o_height, 'iscrowd': 0, 'image_id':
                   image_id, 'bbox':[xmin, ymin, o_width, o_height],
                   'category_id': category_id, 'id': bnd_id, 'ignore': 0,
                   'segmentation': []}
            json_dict['annotations'].append(ann)
            bnd_id = bnd_id + 1

    for cate, cid in categories.items():
        cat = {'supercategory': 'none', 'id': cid, 'name': cate}
        json_dict['categories'].append(cat)
    json_fp = open(json_file, 'w')
    json_str = json.dumps(json_dict)
    json_fp.write(json_str)
    json_fp.close()
    list_fp.close()



In [6]:
if __name__ == '__main__':
    if len(sys.argv) <= 1:
        print('3 auguments are need.')
        print('Usage: %s XML_LIST.txt XML_DIR OUTPU_JSON.json'%(sys.argv[0]))
        exit(1)

    convert("D:\\OCR\\id\\data_original\\xml\\coco_train.txt", "D:\\OCR\\id\\data_original\\xml\\xml_train", "D:\\OCR\\id\\data_original\\xml\\coco_train.json")

Processing 2020_07_10_01_19_23.xml
Processing 2020_07_10_01_32_12.xml
Processing 2020_07_10_01_34_19.xml
Processing 2020_07_10_01_36_28.xml
Processing 2020_07_10_01_40_58.xml
Processing 2020_07_10_01_48_18.xml
Processing 2020_07_10_01_55_56.xml
Processing 2020_07_10_02_03_29.xml
Processing 2020_07_10_02_04_51.xml
Processing 2020_07_10_02_07_20.xml
Processing 2020_07_10_02_19_15.xml
Processing 2020_07_10_02_20_36.xml
Processing 2020_07_10_02_22_32.xml
Processing 2020_07_10_02_25_59.xml
Processing 2020_07_10_02_38_07.xml
Processing 2020_07_10_02_42_20.xml
Processing 2020_07_10_02_47_38.xml
Processing 2020_07_10_02_52_53.xml
Processing 2020_07_10_02_55_32.xml
Processing 2020_07_10_03_02_10.xml
Processing 2020_07_10_03_09_17.xml
Processing 2020_07_10_03_26_19.xml
Processing 2020_07_10_03_32_43.xml
Processing 2020_07_10_03_37_47.xml
Processing 2020_07_10_03_38_33.xml
Processing 2020_07_10_03_45_00.xml
Processing 2020_07_10_04_00_25.xml
Processing 2020_07_10_04_03_20.xml
Processing 2020_07_1

Processing 2020_07_11_21_48_52.xml
Processing 2020_07_11_21_53_57.xml
Processing 2020_07_11_21_54_58.xml
Processing 2020_07_11_23_12_00.xml
Processing 2020_07_11_23_30_10.xml
Processing 2020_07_11_23_31_39.xml
Processing 2020_07_11_23_37_46.xml
Processing 2020_07_11_23_49_16.xml
Processing 2020_07_11_23_56_07.xml
Processing 2020_07_11_23_56_52.xml
Processing 2020_07_11_23_58_24.xml
Processing 2020_07_11_23_59_14.xml
Processing 2020_07_12_00_03_34.xml
Processing 2020_07_12_00_05_56.xml
Processing 2020_07_12_00_08_17.xml
Processing 2020_07_12_00_12_20.xml
Processing 2020_07_12_00_23_43.xml
Processing 2020_07_12_00_27_00.xml
Processing 2020_07_12_00_28_08.xml
Processing 2020_07_12_00_59_06.xml
Processing 2020_07_12_01_02_39.xml
Processing 2020_07_12_01_03_41.xml
Processing 2020_07_12_01_06_32.xml
Processing 2020_07_12_01_07_32.xml
Processing 2020_07_12_01_19_43.xml
Processing 2020_07_12_01_26_08.xml
Processing 2020_07_12_01_35_15.xml
Processing 2020_07_12_01_36_04.xml
Processing 2020_07_1

Processing 2020_07_13_13_04_31.xml
Processing 2020_07_13_13_06_42.xml
Processing 2020_07_13_13_12_03.xml
Processing 2020_07_13_13_13_14.xml
Processing 2020_07_13_13_17_52.xml
Processing 2020_07_13_13_29_42.xml
Processing 2020_07_13_13_32_56.xml
Processing 2020_07_13_13_48_49.xml
Processing 2020_07_13_13_51_08.xml
Processing 2020_07_13_13_52_02.xml
Processing 2020_07_13_13_58_54.xml
Processing 2020_07_13_14_17_57.xml
Processing 2020_07_13_14_21_57.xml
Processing 2020_07_13_14_34_47.xml
Processing 2020_07_13_14_57_43.xml
Processing 2020_07_13_15_13_26.xml
Processing 2020_07_13_15_15_29.xml
Processing 2020_07_13_15_16_37.xml
Processing 2020_07_13_15_17_32.xml
Processing 2020_07_13_15_24_59.xml
Processing 2020_07_13_15_28_15.xml
Processing 2020_07_13_15_29_24.xml
Processing 2020_07_13_15_32_05.xml
Processing 2020_07_13_15_45_06.xml
Processing 2020_07_13_15_48_32.xml
Processing 2020_07_13_15_50_12.xml
Processing 2020_07_13_15_51_51.xml
Processing 2020_07_13_15_55_03.xml
Processing 2020_07_1

Processing 2020_07_14_23_55_24.xml
Processing 2020_07_15_00_08_28.xml
Processing 2020_07_15_00_09_55.xml
Processing 2020_07_15_00_19_21.xml
Processing 2020_07_15_00_20_07.xml
Processing 2020_07_15_00_24_19.xml
Processing 2020_07_15_00_25_09.xml
Processing 2020_07_15_00_26_11.xml
Processing 2020_07_15_00_36_51.xml
Processing 2020_07_15_00_44_10.xml
Processing 2020_07_15_00_45_37.xml
Processing 2020_07_15_00_56_30.xml
Processing 2020_07_15_00_57_36.xml
Processing 2020_07_15_00_58_38.xml
Processing 2020_07_15_00_59_51.xml
Processing 2020_07_15_01_01_33.xml
Processing 2020_07_15_01_06_27.xml
Processing 2020_07_15_01_10_20.xml
Processing 2020_07_15_01_12_03.xml
Processing 2020_07_15_01_13_55.xml
Processing 2020_07_15_01_16_47.xml
Processing 2020_07_15_01_24_18.xml
Processing 2020_07_15_01_25_18.xml
Processing 2020_07_15_01_27_57.xml
Processing 2020_07_15_01_39_03.xml
Processing 2020_07_15_01_41_58.xml
Processing 2020_07_15_01_43_02.xml
Processing 2020_07_15_01_44_02.xml
Processing 2020_07_1

NotImplementedError: Filename NG-007370-(02-22-35)-1T134-1031-(1T134-10n1) is supposed to be an integer.

## 跑資料夾

In [1]:
# 改檔名

import os
path='D:\\OCR\\id\\data_original\\xml\\xml_train' #這就是欲進行檔名更改的檔案路徑，路徑的斜線是為/，要留意下！
files=os.listdir(path)
print('files')    #印出讀取到的檔名稱，用來確認自己是不是真的有讀到


files


In [2]:
n=0 #設定初始值
f = open('D:\\OCR\\id\\data_original\\xml\\coco_train.txt','a')
for i in files: #因為資料夾裡面的檔案都要重新更換名稱
    f.write(i)
    f.write('\n')
f.close()
    